# Get list of users from Active Directory and write to csv

In [1]:
import csv
from pyad import adsearch
from datetime import datetime
from os import listdir

### Functions

In [2]:
import csv
from pyad import adsearch

def extrFirstLastName(displayName):
        firstName, lastName = '', ''
    
        for namePart in displayName.split(' '):
            if namePart == namePart.upper():
                lastName += namePart+' '
            else:
                firstName += namePart+' '
                
        firstName, lastName = firstName.strip(), lastName.strip()
        
        return [firstName, lastName]
    

def extrManagerName(manager):
    for item in manager.split(','):
        if item[:3] == 'CN=':
            manager = item[3:]
            return manager
    return ''


def newUsers(augUsers, lastUsers):
    newUserL = []
    for currentUser in augUsers:
        if currentUser['displayName'] not in [item['displayName'] for item in lastUsers]:
            newUserL.append({'name': currentUser['displayName']})
            
    if newUserL:
        fileName = 'new_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, newUserL)
    
    return


def leftUsers(augUsers, lastUsers):
    leftUserL = []
    for oldUser in lastUsers:
        if oldUser['displayName'] not in [item['displayName'] for item in augUsers]:
            leftUserL.append({'name':oldUser['displayName']})
            
    if leftUserL:
        fileName = 'left_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, leftUserL)
            
    return


def newManagers(augUsers, lastUsers):
    newManagerL = []
    for currentUser in augUsers:
        currentManager = currentUser['manager']
        oldManagerL = [item['manager'] for item in lastUsers if currentUser['displayName'] == item['displayName']]

        if oldManagerL: 
            oldManager = oldManagerL[0]
        else:
            oldManager = ''

        if currentManager != oldManager and currentManager and oldManager:
            newManagerL.append({'name':currentUser['displayName'], 
                                'oldManager':extrManagerName(oldManager), 
                                'newManager':extrManagerName(currentManager) })
    if newManagerL:
        fileName = 'newMngr_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, newManagerL)
    
    return

# - - - - - -

def writeCSV(fileName, listOfDict):
    attributeList = list(listOfDict[-1].keys())
    csv.register_dialect('myDialect', delimiter = ',' , lineterminator = '\n', quoting=csv.QUOTE_ALL)

    print('... writing {}'.format(fileName))

    with open(fileName, mode='w') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=attributeList, dialect='myDialect')
        writer.writeheader()
        writer.writerows(listOfDict)   

    csv_file.close()
    return

### get users from AD

In [3]:
print('= = = Get Users From AD = = =')
print('... getting users ...')

q = adsearch.ADQuery()
attributeList = ['displayName', 'mail', 'title', 'telephoneNumber', 'mobile', 'manager', 'directReports', 'physicalDeliveryOfficeName']
q.execute_query( attributes=attributeList, where_clause = "objectClass = '*'", base_dn = "OU=Users,OU=AUG,OU=DE,DC=ls,DC=ege,DC=ds")

augUsers = [ row for row in q.get_results() if row['displayName']  ]

= = = Get Users From AD = = =
... getting users ...


### name processing

In [5]:
for i,user in enumerate(augUsers):
    augUsers[i]['firstName'], augUsers[i]['lastName'] = extrFirstLastName(user['displayName'])
    
    if user['manager']:
        managerName = extrManagerName(user['manager'])
        augUsers[i]['managerName']= managerName

### read recent users from csv

In [7]:
print('... comparing to recent users')

lastfilesL = [item for item in listdir('./') if 'augUsers' == item[:8]]

if lastfilesL:
    lastfile = sorted(lastfilesL)[-1]
    print('... comparing to file: {}'.format(lastfile))

    with open(lastfile) as f:
         lastUsers = [{k: v for k, v in row.items()}
              for row in csv.DictReader(f, skipinitialspace=True)]

... comparing to recent users
... comparing to file: augUsers190527_134653.csv


### users with new manager

In [9]:
newManagerL = []
for currentUser in augUsers:
    currentManager = currentUser['manager']
    oldManagerL = [item['manager'] for item in lastUsers if currentUser['displayName'] == item['displayName']]

    if oldManagerL: 
        oldManager = oldManagerL[0]
    else:
        oldManager = ''

    if currentManager != oldManager and currentManager and oldManager:
        newManagerL.append({'name':currentUser['displayName'], 
                            'oldManager':extrManagerName(oldManager), 
                            'newManager':extrManagerName(currentManager) })

### users left

In [12]:
leftUserL = []
for oldUser in lastUsers:
    if oldUser['displayName'] not in [item['displayName'] for item in augUsers]:
        leftUserL.append({'name':oldUser['displayName']})

### new users

In [17]:
newUserL = []
for currentUser in augUsers:
    if currentUser['displayName'] not in [item['displayName'] for item in lastUsers]:
        newUserL.append({'name': currentUser['displayName']})